# Assignment 3a: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

In this assignment, you'll build a simple Gradio frontend for your RAG system with just the essential features:
- Button to initialize the vector database
- Search query input and button
- Display of AI responses

**Learning Objectives:**
- Create basic Gradio interfaces
- Connect RAG backend to frontend
- Handle user interactions and database initialization
- Build functional AI-powered web applications

**Prerequisites:**
- Completed Assignment 1 (Vector Database Basics)
- Completed Assignment 2 (Advanced RAG)
- Understanding of LlamaIndex fundamentals

---
## 🔑 Setup: Configure Your API Key

**This assignment uses OpenRouter** (cheaper alternative to OpenAI direct).

### Get Your OpenRouter API Key:
1. Go to: https://openrouter.ai/keys
2. Sign up or log in (supports Google sign-in)
3. Create a new API key
4. Copy the key (starts with `sk-or-v1-...`)

### Why OpenRouter?
- ✅ Access to multiple models (GPT-4, Claude, etc.)
- ✅ Often cheaper than direct OpenAI access
- ✅ Easy to compare models
- ✅ Good for learning

### Cost Estimate:
- Using GPT-4o-mini via OpenRouter
- This assignment: ~5-10 queries = **$0.005 - $0.01 total**
- Very affordable!

**Alternative:** You can also use OpenAI API key directly if you prefer.

In [ ]:
# API Key Configuration
import os
from getpass import getpass

# Check if API key is already set
if not os.getenv("OPENROUTER_API_KEY") and not os.getenv("OPENAI_API_KEY"):
    print("\n🔑 API Key Configuration")
    print("=" * 50)
    print("This assignment needs an LLM API key.\n")
    print("Option 1 (Recommended): OpenRouter API key")
    print("  Get from: https://openrouter.ai/keys")
    print("  Format: sk-or-v1-...\n")
    print("Option 2: OpenAI API key")
    print("  Get from: https://platform.openai.com/api-keys")
    print("  Format: sk-proj-... or sk-...\n")
    
    api_key = getpass("Paste your API key: ").strip()
    
    if api_key:
        if api_key.startswith("sk-or-"):
            os.environ["OPENROUTER_API_KEY"] = api_key
            print("\n✅ OpenRouter API key configured!")
        elif api_key.startswith("sk-"):
            os.environ["OPENAI_API_KEY"] = api_key
            print("\n✅ OpenAI API key configured!")
        else:
            print("\n⚠️  Warning: API key format not recognized. Setting as OpenRouter key.")
            os.environ["OPENROUTER_API_KEY"] = api_key
    else:
        print("\n⚠️  No API key entered. Please run this cell again.")
else:
    print("✅ API key already configured!")

---
## 📚 Part 1: Setup and Imports

**What this does:**
- Imports Gradio for building the web UI
- Imports LlamaIndex components (same as Assignments 1 & 2)
- Imports OpenRouter for LLM operations

**New Library - Gradio:**
- Python library for building ML/AI web interfaces
- Simple API (just Python, no HTML/CSS/JS needed)
- Automatic UI generation from function signatures
- Built-in hosting (can share publicly)

**Why Gradio?**
- ✅ Fast prototyping (minutes, not hours)
- ✅ No frontend coding required
- ✅ Works in Jupyter notebooks
- ✅ Easy to share demos

In [ ]:
# Import required libraries
import gradio as gr
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")

---
## 🤖 Part 2: RAG Backend Class

**What this class does:**
- Wraps your RAG system in a reusable class
- Handles database initialization
- Processes user queries
- Manages errors gracefully

**Key Methods:**
1. `__init__()`: Initialize with settings
2. `setup_settings()`: Configure LlamaIndex (LLM + embeddings)
3. `initialize_database()`: Load documents, create vector index
4. `query()`: Answer user questions using RAG

**Design Pattern - Backend/Frontend Separation:**
- **Backend** (this class): Business logic, data processing
- **Frontend** (next cell): User interface, interactions
- **Benefit**: Can swap UI (Gradio → Streamlit → Flask) without changing backend

**Error Handling:**
- Returns user-friendly messages instead of crashing
- Checks for common issues (missing data, no index, empty query)
- Uses try/except for robustness

In [ ]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""
    
    def __init__(self):
        self.index = None
        self.setup_settings()
    
    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Try OpenRouter first, fall back to OpenAI
        openrouter_key = os.getenv("OPENROUTER_API_KEY")
        openai_key = os.getenv("OPENAI_API_KEY")
        
        if openrouter_key:
            from llama_index.llms.openrouter import OpenRouter
            Settings.llm = OpenRouter(
                api_key=openrouter_key,
                model="openai/gpt-4o-mini",  # Via OpenRouter
                temperature=0.1
            )
            print("✅ Using OpenRouter for LLM (gpt-4o-mini)")
        elif openai_key:
            from llama_index.llms.openai import OpenAI
            Settings.llm = OpenAI(
                api_key=openai_key,
                model="gpt-4o-mini",
                temperature=0.1
            )
            print("✅ Using OpenAI for LLM (gpt-4o-mini)")
        else:
            print("⚠️  No API key found - LLM operations will fail")
        
        # Set up the embedding model (local, free)
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )
        
        # Set chunking parameters
        Settings.chunk_size = 512
        Settings.chunk_overlap = 50
        print("✅ Local embeddings configured (BAAI/bge-small-en-v1.5)")
    
    def initialize_database(self, data_folder="data"):
        """Initialize the vector database with documents."""
        # Check if data folder exists
        if not Path(data_folder).exists():
            return f"❌ Data folder '{data_folder}' not found! Please check the path."
        
        try:
            # Create vector store
            vector_store = LanceDBVectorStore(
                uri="./basic_rag_vectordb",
                table_name="documents"
            )
            
            # Load documents
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()
            
            if len(documents) == 0:
                return f"❌ No documents found in '{data_folder}'!"
            
            # Create storage context and index
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents, 
                storage_context=storage_context,
                show_progress=True
            )
            
            return f"✅ Database initialized successfully with {len(documents)} documents!"
        
        except Exception as e:
            return f"❌ Error initializing database: {str(e)}"
    
    def query(self, question):
        """Query the RAG system and return response."""
        # Check if index exists
        if self.index is None:
            return "❌ Please initialize the database first! Click the 'Initialize Database' button above."
        
        # Check if question is empty
        if not question or not question.strip():
            return "⚠️ Please enter a question first!"
        
        try:
            # Create query engine and get response
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            return str(response)
        
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
rag_backend = SimpleRAGBackend()
print("\n🚀 RAG Backend initialized and ready!")

---
## 🎨 Part 3: Gradio Interface (TODO - Complete This Section)

**What you'll build:**
A simple web interface with these components:
1. **Title** - Using `gr.Markdown()`
2. **Initialize Button** - Using `gr.Button()`
3. **Status Output** - Using `gr.Textbox()`
4. **Query Input** - Using `gr.Textbox()`
5. **Submit Button** - Using `gr.Button()`
6. **Response Output** - Using `gr.Textbox()`

**Gradio Basics:**

### Creating a Layout:
```python
with gr.Blocks() as interface:
    # Add components here
    title = gr.Markdown("# My App")
    button = gr.Button("Click Me")
```

### Component Types:
```python
# Display text/markdown
gr.Markdown("# Title")

# Button
btn = gr.Button("Button Label")

# Text input
input_box = gr.Textbox(label="Enter text", placeholder="Type here...")

# Text output (read-only)
output_box = gr.Textbox(label="Output", interactive=False)
```

### Connecting Components:
```python
# Connect button to function
btn.click(
    fn=my_function,      # Function to call
    inputs=[input_box],  # What to pass as input
    outputs=[output_box] # Where to put the result
)
```

**Your Task:**
Complete the TODOs below to create the interface!

In [ ]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""
    
    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database()
    
    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)
    
    # Create Gradio interface using gr.Blocks()
    with gr.Blocks(title="Basic RAG Assistant") as interface:
        
        # TODO 1: Add title and description
        # Hint: Use gr.Markdown() for formatted text
        # Example: gr.Markdown("# My Title")
        gr.Markdown("# 🤖 Basic RAG Assistant")
        gr.Markdown("Ask questions about your documents using AI-powered search!")
        
        # Add some space
        gr.Markdown("---")
        
        # TODO 2: Add initialization section
        # Hint: Create a button with gr.Button("Button Text")
        gr.Markdown("### Step 1: Initialize Database")
        gr.Markdown("Click the button below to load documents and create the vector database.")
        init_btn = gr.Button("🚀 Initialize Database", variant="primary")
        
        # TODO 3: Add status output
        # Hint: Use gr.Textbox(label="Status", interactive=False) for read-only output
        status_output = gr.Textbox(
            label="Status",
            placeholder="Click 'Initialize Database' to start...",
            interactive=False,
            lines=2
        )
        
        # Add some space
        gr.Markdown("---")
        
        # TODO 4: Add query section
        # Hint: You need:
        # - gr.Textbox() for input (with placeholder)
        # - gr.Button() for submit
        # - gr.Textbox() for response output (interactive=False)
        
        gr.Markdown("### Step 2: Ask Questions")
        gr.Markdown("Enter your question below and click 'Ask Question' to get an AI-powered answer.")
        
        query_input = gr.Textbox(
            label="Your Question",
            placeholder="What would you like to know about the documents?",
            lines=2
        )
        
        submit_btn = gr.Button("💬 Ask Question", variant="primary")
        
        response_output = gr.Textbox(
            label="AI Response",
            placeholder="Response will appear here...",
            interactive=False,
            lines=10
        )
        
        # TODO 5: Connect buttons to functions
        # Hint: Use button.click(function, inputs=[...], outputs=[...])
        # Example: btn.click(fn=my_func, inputs=[input_box], outputs=[output_box])
        
        # Connect initialize button
        init_btn.click(
            fn=initialize_db,
            inputs=None,  # No inputs needed
            outputs=[status_output]
        )
        
        # Connect submit button
        submit_btn.click(
            fn=handle_query,
            inputs=[query_input],
            outputs=[response_output]
        )
        
    return interface

# Create the interface
print("🎨 Creating Gradio interface...")
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")
print("\n💡 Run the next cell to launch the app!")

---
## 🚀 Part 4: Launch Your Application

**What this does:**
- Starts a local web server
- Opens your app in a browser
- Makes your RAG system accessible via web UI

**Launch Options:**
```python
# Basic launch (default: localhost:7860)
interface.launch()

# Custom port
interface.launch(server_port=8080)

# Public URL (shareable link - 72 hours)
interface.launch(share=True)

# Inline in Jupyter
interface.launch(inline=True)
```

**Testing Instructions:**
1. **Initialize Database** - Click the button and wait for success message (~30-60 seconds)
2. **Ask Questions** - Try the example questions below
3. **Experiment** - Try different queries to test semantic search

**Example Questions:**
- "What are the main topics in the documents?"
- "Summarize the key findings"
- "What are AI agents and how do they work?"
- "Explain the methodology used in the research"

**Troubleshooting:**
- **Database init fails**: Check that `data` folder exists with documents
- **Query fails**: Make sure database was initialized first
- **Slow responses**: Normal for first query (model loading), faster after

In [ ]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message (~30-60 seconds)")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- What are AI agents and how do they work?")
print("- Explain the methodology used")
print("")
print("🚀 Launching app...")
print("")

# Launch the application
# Default: Opens at http://localhost:7860
basic_interface.launch(
    server_port=7860,  # Default Gradio port
    share=False,       # Set to True for public URL (expires in 72 hours)
    inline=False       # Set to True to display inline in Jupyter
)

---
## ✅ Assignment Completion Checklist

Before submitting, ensure you have:

### Implementation:
- [x] RAG backend is provided and working
- [x] Created Gradio interface with required components:
  - [x] Title and description using `gr.Markdown()`
  - [x] Initialize database button using `gr.Button()`
  - [x] Status output using `gr.Textbox()`
  - [x] Query input field using `gr.Textbox()`
  - [x] Submit query button using `gr.Button()`
  - [x] Response output area using `gr.Textbox()`
- [x] Connected buttons to backend functions using `.click()`
- [x] Successfully launched the application

### Testing:
- [ ] Tested database initialization (should show success message)
- [ ] Tested query functionality (should return AI responses)
- [ ] Tested error handling (try querying before initialization)
- [ ] Tested with multiple different questions

### Understanding:
- [ ] Understand how Gradio components work
- [ ] Understand how to connect UI to backend functions
- [ ] Understand the RAG query flow (database → retrieval → LLM → response)

---

## 🎊 Congratulations!

You've successfully built your first Gradio RAG application! You now have:

✅ **A functional web interface** for your RAG system
✅ **Understanding of Gradio basics** and component connections
✅ **A foundation** for building more complex AI applications
✅ **Hands-on experience** with frontend-backend integration

### What You Learned:
- Creating web UIs with Gradio (no HTML/CSS/JS needed!)
- Connecting UI components to Python functions
- Building interactive AI applications
- Handling user interactions and errors
- Deploying RAG systems with web interfaces

### Next Steps:
**Complete Assignment 3b** to add advanced configuration options:
- API key input in the UI
- Adjustable similarity threshold
- Different response modes
- Model selection

---

## 📚 Additional Resources:
- Gradio Docs: https://www.gradio.app/docs
- LlamaIndex Docs: https://docs.llamaindex.ai/
- OpenRouter: https://openrouter.ai/docs